In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model

2024-06-02 12:51:08.890562: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-02 12:51:10.437208: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-02 12:51:10.437252: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-02 12:51:10.442118: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-02 12:51:11.103564: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-02 12:51:11.104804: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
# Set the paths to the dataset directories
train_dir = 'dataset/train'
test_dir = 'dataset/test'
val_dir = 'dataset/val'

In [3]:
# Define the image dimensions and batch size
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32

In [4]:
# Create ImageDataGenerators for training, test, and validation sets
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='input',  # Use 'input' mode for autoencoder
    shuffle=True
)


Found 40982 images belonging to 5 classes.


In [6]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='input',  # Use 'input' mode for autoencoder
    shuffle=False  # No need to shuffle test data
)

Found 11714 images belonging to 5 classes.


In [7]:
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='input',  # Use 'input' mode for autoencoder
    shuffle=False  # No need to shuffle validation data
)

Found 5853 images belonging to 5 classes.


In [8]:
# Define the input shape
input_shape = train_generator.image_shape

In [9]:
# Define the input tensor
input_img = Input(shape=input_shape)


In [10]:
# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
encoded = Conv2D(128, (3, 3), activation='relu', padding='same')(x)


In [11]:
# Decoder
x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)


In [12]:
# Create the Autoencoder model
autoencoder = Model(input_img, decoded)

In [13]:
# Compile the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')


In [14]:
# Train the Autoencoder
history = autoencoder.fit(
    train_generator,
    epochs=20,
    batch_size=BATCH_SIZE,
    validation_data=val_generator
)

Epoch 1/20
1281/1281 [==============================] - 1038s 808ms/step - loss: 0.0021 - val_loss: 7.9496e-04
Epoch 2/20
1281/1281 [==============================] - 1029s 803ms/step - loss: 7.3837e-04 - val_loss: 6.8797e-04
Epoch 3/20
1281/1281 [==============================] - 1027s 802ms/step - loss: 6.1062e-04 - val_loss: 5.6369e-04
Epoch 4/20
1281/1281 [==============================] - 1026s 801ms/step - loss: 5.5227e-04 - val_loss: 4.9225e-04
Epoch 5/20
1281/1281 [==============================] - 1019s 795ms/step - loss: 5.0064e-04 - val_loss: 4.2752e-04
Epoch 6/20
1281/1281 [==============================] - 1039s 811ms/step - loss: 4.6980e-04 - val_loss: 4.4478e-04
Epoch 7/20
1281/1281 [==============================] - 1017s 794ms/step - loss: 4.3852e-04 - val_loss: 4.1991e-04
Epoch 8/20
1281/1281 [==============================] - 1022s 798ms/step - loss: 4.1642e-04 - val_loss: 8.1731e-04
Epoch 9/20
1281/1281 [==============================] - 1016s 793ms/step - loss: 3.9

In [15]:
# Evaluate the Autoencoder on the test set
test_loss = autoencoder.evaluate(test_generator)


367/367 [==============================] - 49s 134ms/step - loss: 2.8548e-04


In [16]:
# Save the trained Autoencoder model
autoencoder.save('../autoencoder_model/autoencoder.keras')
